In [257]:
import os
import requests
import h3
import geopy.distance
import geopy.point

import pandas as pd
import numpy as np

from dotenv import load_dotenv


load_dotenv()

True

In [271]:
# Fix a seed for sampling and preparation
seed = 42
sample_size = 10

cols = [
    'geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude',
    'feature_class', 'feature_code', 'country_code', 'cc2', 'admin1_code',
    'admin2_code', 'admin3_code', 'admin4_code', 'population', 'elevation',
    'dem', 'timezone', 'modification_date'
]

# Load the data
df = pd.read_csv(
    '/Users/kaimoltzen/Downloads/cities15000.txt',
    sep='\t',
    header=None,
    names=cols,
    usecols=['geonameid', 'name', 'latitude', 'longitude', 'feature_code', 'population', 'country_code'],
    low_memory=False,
    encoding='utf-8'
)

# Only US cities
df = df[df['country_code'] == 'US']

# Sample x rows
sample = df.sample(sample_size, random_state=seed)
head = sample.head(10)
# print the head of the sample and make sure all columns are printed
pd.set_option('display.max_columns', None)
sample

,geonameid,name,latitude,longitude,feature_code,country_code,population
26754,4745272,Baileys Crossroads,38.85039,-77.12970,PPL,US,23643
27996,5253352,Fond du Lac,43.77500,-88.43883,PPLA2,US,42933
26445,4529987,Bartlesville,36.74731,-95.98082,PPLA2,US,36595
28338,5378538,Oakland,37.80437,-122.27080,PPLA2,US,419267
25834,4165565,Naples,26.14234,-81.79596,PPLA2,US,21512
28266,5364199,Ladera Ranch,33.57086,-117.63561,PPL,US,22980
28649,5656882,Helena,46.59271,-112.03611,PPLA,US,32091
27445,5055787,Kirksville,40.19475,-92.58325,PPLA2,US,17520
28202,5344994,East Los Angeles,34.02390,-118.17202,PPL,US,126496
27648,5118226,Garden City,40.72677,-73.63430,PPL,US,22612


In [273]:
geospatial_prep_with_bearing = {
    'north': 0,
    'northeast': 45,
    'east': 90,
    'southeast': 135,
    'south': 180,
    'southwest': 225,
    'west': 270,
    'northwest': 315,
    'between': None,
    'near': None
}

In [274]:
# Extend the dataframe with geospatial prepositions, bearing, distance, and H3 cell resolution
rng = np.random.default_rng(seed)
# 1. randomly choose a geospatial preposition from the dictionary and add it in a new column
preps = list(geospatial_prep_with_bearing.keys())
sample["geospatial_prep"] = rng.choice(preps, size=sample_size)

# 2. Choose the corresponding bearing from the dictionary and add it in a new column. For near, choose a random number between 0 and 360 as bearing. For between, use the GeoNames API to get a city >15k population  nearby
sample["bearing"] = sample["geospatial_prep"].map(geospatial_prep_with_bearing)
sample["bearing"] = sample.apply(
    lambda x: rng.integers(0, 361) if x["geospatial_prep"] == "near" else x["bearing"],
    axis=1
)

def get_nearby_city(latitude, longitude, population):
    response = requests.get(
        f"http://api.geonames.org/findNearbyPlaceNameJSON?lat={latitude}&lng={longitude}&cities=cities15000&radius={population/1000}&maxRows=10&username={os.getenv("GEONAMES_USERNAME")}"
    )
    if response.status_code == 200:
        data = response.json()
        if data['geonames']:
            nearby_city = data['geonames'][1] if len(data['geonames']) > 1 else None # 0 is the same city
            return nearby_city
    return None

sample["between_object"] = sample.apply(
    lambda x: get_nearby_city(x["latitude"], x["longitude"], x["population"]) if x["geospatial_prep"] == "between" else None,
    axis=1
)

# 3. # For directions: Distance is sampled from a log-normal distribution with mean log(population/5) and std 1
sample["distance"] = sample.apply(
    lambda x: rng.lognormal(mean=np.log(x["population"] / 5), sigma=1) if x["geospatial_prep"] != "between" else None,
    axis=1
)

# 4. Generate the target location description
def generate_target_location_description(row):
    geospatial_prep = row['geospatial_prep']
    name = row['name']
    if geospatial_prep == "between":
        return f"{geospatial_prep} {name} and {row['between_object']['name']}"
    elif geospatial_prep == "near":
        return f"{geospatial_prep} {name}"
    else:
        return f"{round(row['distance'] / 1000, 1)} km {geospatial_prep} of {name}"

sample['target_location_description'] = sample.apply(generate_target_location_description, axis=1)
sample

,geonameid,name,latitude,longitude,feature_code,country_code,population,geospatial_prep,bearing,between_object,distance,target_location_description
26754,4745272,Baileys Crossroads,38.85039,-77.12970,PPL,US,23643,north,0.0,None,1285.888178,1.3 km north of Baileys Crossroads
27996,5253352,Fond du Lac,43.77500,-88.43883,PPLA2,US,42933,northwest,315.0,None,9757.568612,9.8 km northwest of Fond du Lac
26445,4529987,Bartlesville,36.74731,-95.98082,PPLA2,US,36595,west,270.0,None,5334.691803,5.3 km west of Bartlesville
28338,5378538,Oakland,37.80437,-122.27080,PPLA2,US,419267,south,180.0,None,82456.334839,82.5 km south of Oakland
25834,4165565,Naples,26.14234,-81.79596,PPLA2,US,21512,south,180.0,None,1833.321005,1.8 km south of Naples
28266,5364199,Ladera Ranch,33.57086,-117.63561,PPL,US,22980,between,NaN,"{'adminCode1': 'CA', 'lng': '-117.672', 'dista...",NaN,between Ladera Ranch and Mission Viejo
28649,5656882,Helena,46.59271,-112.03611,PPLA,US,32091,north,0.0,None,15464.323781,15.5 km north of Helena
27445,5055787,Kirksville,40.19475,-92.58325,PPLA2,US,17520,west,270.0,None,7627.019261,7.6 km west of Kirksville
28202,5344994,East Los Angeles,34.02390,-118.17202,PPL,US,126496,east,90.0,None,27026.110986,27.0 km east of East Los Angeles
27648,5118226,Garden City,40.72677,-73.63430,PPL,US,22612,north,0.0,None,13961.227641,14.0 km north of Garden City


In [275]:
sample["target_center"] = sample.apply(
    lambda x: geopy.distance.distance(kilometers=x["distance"]/1000).destination(
        (x["latitude"], x["longitude"]),
        x["bearing"]
    )if x["geospatial_prep"] != "between" else geopy.point.Point(
        latitude=(float(x["between_object"]["lat"]) + float(x["latitude"])) / 2,
        longitude=(float(x["between_object"]["lng"]) + float(x["longitude"])) / 2,
    ),
    axis=1
)
sample

,geonameid,name,latitude,longitude,feature_code,country_code,population,geospatial_prep,bearing,between_object,distance,target_location_description,target_center
26754,4745272,Baileys Crossroads,38.85039,-77.12970,PPL,US,23643,north,0.0,None,1285.888178,1.3 km north of Baileys Crossroads,"38 51m 43.1037s N, 77 7m 46.92s W"
27996,5253352,Fond du Lac,43.77500,-88.43883,PPLA2,US,42933,northwest,315.0,None,9757.568612,9.8 km northwest of Fond du Lac,"43 50m 13.4379s N, 88 31m 28.6298s W"
26445,4529987,Bartlesville,36.74731,-95.98082,PPLA2,US,36595,west,270.0,None,5334.691803,5.3 km west of Bartlesville,"36 44m 50.262s N, 96 2m 25.9994s W"
28338,5378538,Oakland,37.80437,-122.27080,PPLA2,US,419267,south,180.0,None,82456.334839,82.5 km south of Oakland,"37 3m 41.1298s N, 122 16m 14.88s W"
25834,4165565,Naples,26.14234,-81.79596,PPLA2,US,21512,south,180.0,None,1833.321005,1.8 km south of Naples,"26 7m 32.8523s N, 81 47m 45.456s W"
28266,5364199,Ladera Ranch,33.57086,-117.63561,PPL,US,22980,between,NaN,"{'adminCode1': 'CA', 'lng': '-117.672', 'dista...",NaN,between Ladera Ranch and Mission Viejo,"33 35m 7.584s N, 117 39m 13.698s W"
28649,5656882,Helena,46.59271,-112.03611,PPLA,US,32091,north,0.0,None,15464.323781,15.5 km north of Helena,"46 43m 54.5606s N, 112 2m 9.996s W"
27445,5055787,Kirksville,40.19475,-92.58325,PPLA2,US,17520,west,270.0,None,7627.019261,7.6 km west of Kirksville,"40 11m 40.9753s N, 92 40m 22.1546s W"
28202,5344994,East Los Angeles,34.02390,-118.17202,PPL,US,126496,east,90.0,None,27026.110986,27.0 km east of East Los Angeles,"34 1m 24.7867s N, 117 52m 45.841s W"
27648,5118226,Garden City,40.72677,-73.63430,PPL,US,22612,north,0.0,None,13961.227641,14.0 km north of Garden City,"40 51m 8.96534s N, 73 38m 3.48s W"


In [276]:
# Randomly choose a H3 cell resolution between 5 and 8 to determine the size of the area
sample["h3_resolution"] = rng.integers(5, 9, size=sample_size)
sample["target_cell"] = sample.apply(
    lambda x: h3.latlng_to_cell(lat=x["target_center"].latitude,
                                lng=x["target_center"].longitude,
                                res=x["h3_resolution"]),
    axis=1
)
sample["target_area"] = sample.apply(
    lambda x: h3.cell_area(x["target_cell"], unit="m^2"),
    axis=1
)
sample

,geonameid,name,latitude,longitude,feature_code,country_code,population,geospatial_prep,bearing,between_object,distance,target_location_description,target_center,h3_resolution,target_cell,target_area
26754,4745272,Baileys Crossroads,38.85039,-77.12970,PPL,US,23643,north,0.0,None,1285.888178,1.3 km north of Baileys Crossroads,"38 51m 43.1037s N, 77 7m 46.92s W",7,872aa84c8ffffff,5.009619e+06
27996,5253352,Fond du Lac,43.77500,-88.43883,PPLA2,US,42933,northwest,315.0,None,9757.568612,9.8 km northwest of Fond du Lac,"43 50m 13.4379s N, 88 31m 28.6298s W",6,86275c367ffffff,3.755665e+07
26445,4529987,Bartlesville,36.74731,-95.98082,PPLA2,US,36595,west,270.0,None,5334.691803,5.3 km west of Bartlesville,"36 44m 50.262s N, 96 2m 25.9994s W",6,8626ee157ffffff,3.866924e+07
28338,5378538,Oakland,37.80437,-122.27080,PPLA2,US,419267,south,180.0,None,82456.334839,82.5 km south of Oakland,"37 3m 41.1298s N, 122 16m 14.88s W",5,85283417fffffff,2.652749e+08
25834,4165565,Naples,26.14234,-81.79596,PPLA2,US,21512,south,180.0,None,1833.321005,1.8 km south of Naples,"26 7m 32.8523s N, 81 47m 45.456s W",5,854413c7fffffff,2.333873e+08
28266,5364199,Ladera Ranch,33.57086,-117.63561,PPL,US,22980,between,NaN,"{'adminCode1': 'CA', 'lng': '-117.672', 'dista...",NaN,between Ladera Ranch and Mission Viejo,"33 35m 7.584s N, 117 39m 13.698s W",7,8729a5492ffffff,5.811146e+06
28649,5656882,Helena,46.59271,-112.03611,PPLA,US,32091,north,0.0,None,15464.323781,15.5 km north of Helena,"46 43m 54.5606s N, 112 2m 9.996s W",8,882792756dfffff,7.022935e+05
27445,5055787,Kirksville,40.19475,-92.58325,PPLA2,US,17520,west,270.0,None,7627.019261,7.6 km west of Kirksville,"40 11m 40.9753s N, 92 40m 22.1546s W",5,85267223fffffff,2.509962e+08
28202,5344994,East Los Angeles,34.02390,-118.17202,PPL,US,126496,east,90.0,None,27026.110986,27.0 km east of East Los Angeles,"34 1m 24.7867s N, 117 52m 45.841s W",8,8829a0a5a1fffff,8.258415e+05
27648,5118226,Garden City,40.72677,-73.63430,PPL,US,22612,north,0.0,None,13961.227641,14.0 km north of Garden City,"40 51m 8.96534s N, 73 38m 3.48s W",8,882a101513fffff,7.423544e+05


In [277]:
# Invoke VLM to generate description of the area
from openai import OpenAI
import io
import base64
import math
import matplotlib.pyplot as plt
import contextily as cx
from pyproj import Transformer, CRS

# API configuration
api_key = '5af4a10a69da94b31a2f6da990fbc4ff'
base_url = "https://chat-ai.academiccloud.de/v1"
model = "gemma-3-27b-it"

client = OpenAI(
    api_key=api_key,
    base_url=base_url,
)


def generate_map_image(
        lat,
        lon,
        map_size_meters=1000,
        image_size_pixels=1024,
        tile_provider=cx.providers.OpenStreetMap.Mapnik
):
    """
    Generates a base64-encoded PNG image of a map centered at the given latitude and longitude,
    approximating the bounding box from meters to lat/lon degrees at that center point.

    Steps:
      1. Convert desired map size (meters) to approximate offsets in latitude/longitude degrees.
         - 1 degree of latitude ~ 111.32 km
         - 1 degree of longitude ~ 111.32 km * cos(latitude)
      2. Build lat/lon bounding box.
      3. Transform corners to EPSG:3857 and set axis bounds for contextily.
      4. Return the map as a base64-encoded PNG string.

    Parameters:
      lat (float): Latitude of the center point (WGS84).
      lon (float): Longitude of the center point (WGS84).
      map_size_meters (float): Desired side length of the map in meters.
      image_size_pixels (int): Output image width/height in pixels.
      tile_provider: Tile provider for the basemap.

    Returns:
      str: Base64-encoded PNG image of the map.
    """

    # Approximate meters per degree of latitude and longitude near this latitude
    # 1 deg latitude ~ 111.32 km
    # 1 deg longitude ~ 111.32 km * cos(latitude)
    meters_per_degree_lat = 111320.0  # approximate
    lat_radians = math.radians(lat)
    meters_per_degree_lon = 111320.0 * math.cos(lat_radians)

    # Convert the desired map size in meters to degrees of lat/lon
    d_lat = (map_size_meters / meters_per_degree_lat) / 2.0
    d_lon = (map_size_meters / meters_per_degree_lon) / 2.0

    # Build the bounding box in lat/lon
    lat_min = lat - d_lat
    lat_max = lat + d_lat
    lon_min = lon - d_lon
    lon_max = lon + d_lon

    # Prepare transformation from WGS84 (EPSG:4326) to Web Mercator (EPSG:3857)
    wgs84 = CRS.from_epsg(4326)
    web_merc = CRS.from_epsg(3857)
    transformer_to_web = Transformer.from_crs(wgs84, web_merc, always_xy=True)

    # Transform bounding box corners to EPSG:3857
    x_min, y_min = transformer_to_web.transform(lon_min, lat_min)
    x_max, y_max = transformer_to_web.transform(lon_max, lat_max)

    # Create figure
    dpi = 100
    fig, ax = plt.subplots(figsize=(image_size_pixels / dpi, image_size_pixels / dpi), dpi=dpi)

    # Set axes bounds in EPSG:3857
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.axis('off')

    # Add basemap (contextily automatically picks a zoom level)
    cx.add_basemap(ax, crs=web_merc.to_string(), source=tile_provider)

    # Save to memory
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png', bbox_inches='tight', pad_inches=0)
    #plt.savefig('map.png', format='png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    buffer.seek(0)

    # Encode image as base64
    image_base64 = base64.b64encode(buffer.read()).decode('utf-8')

    response = client.chat.completions.create(
    model=model,
    messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Please provide a concise description of this area. Start with 'The area is ...'",
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{image_base64}",
                        },
                    },
                ],
            }
        ],
    )
    return response.choices[0].message.content

sample["target_area_description"] = sample.apply(
    lambda x: generate_map_image(lat=x["target_center"].latitude,
                                 lon=x["target_center"].longitude,
                                 map_size_meters=2 * np.sqrt(x["target_area"])),
    axis=1
)

In [278]:
sample

,geonameid,name,latitude,longitude,feature_code,country_code,population,geospatial_prep,bearing,between_object,distance,target_location_description,target_center,h3_resolution,target_cell,target_area,target_area_description
26754,4745272,Baileys Crossroads,38.85039,-77.12970,PPL,US,23643,north,0.0,None,1285.888178,1.3 km north of Baileys Crossroads,"38 51m 43.1037s N, 77 7m 46.92s W",7,872aa84c8ffffff,5.009619e+06,The area is a moderately dense urban and subur...
27996,5253352,Fond du Lac,43.77500,-88.43883,PPLA2,US,42933,northwest,315.0,None,9757.568612,9.8 km northwest of Fond du Lac,"43 50m 13.4379s N, 88 31m 28.6298s W",6,86275c367ffffff,3.755665e+07,"The area is a mix of rural farmland, wildlife ..."
26445,4529987,Bartlesville,36.74731,-95.98082,PPLA2,US,36595,west,270.0,None,5334.691803,5.3 km west of Bartlesville,"36 44m 50.262s N, 96 2m 25.9994s W",6,8626ee157ffffff,3.866924e+07,The area is a mix of rural farmland and small ...
28338,5378538,Oakland,37.80437,-122.27080,PPLA2,US,419267,south,180.0,None,82456.334839,82.5 km south of Oakland,"37 3m 41.1298s N, 122 16m 14.88s W",5,85283417fffffff,2.652749e+08,The area is a coastal region in California fea...
25834,4165565,Naples,26.14234,-81.79596,PPLA2,US,21512,south,180.0,None,1833.321005,1.8 km south of Naples,"26 7m 32.8523s N, 81 47m 45.456s W",5,854413c7fffffff,2.333873e+08,The area is a coastal region of Southwest Flor...
28266,5364199,Ladera Ranch,33.57086,-117.63561,PPL,US,22980,between,NaN,"{'adminCode1': 'CA', 'lng': '-117.672', 'dista...",NaN,between Ladera Ranch and Mission Viejo,"33 35m 7.584s N, 117 39m 13.698s W",7,8729a5492ffffff,5.811146e+06,The area is a suburban residential zone center...
28649,5656882,Helena,46.59271,-112.03611,PPLA,US,32091,north,0.0,None,15464.323781,15.5 km north of Helena,"46 43m 54.5606s N, 112 2m 9.996s W",8,882792756dfffff,7.022935e+05,The area is a suburban residential neighborhoo...
27445,5055787,Kirksville,40.19475,-92.58325,PPLA2,US,17520,west,270.0,None,7627.019261,7.6 km west of Kirksville,"40 11m 40.9753s N, 92 40m 22.1546s W",5,85267223fffffff,2.509962e+08,The area is a largely rural region in northeas...
28202,5344994,East Los Angeles,34.02390,-118.17202,PPL,US,126496,east,90.0,None,27026.110986,27.0 km east of East Los Angeles,"34 1m 24.7867s N, 117 52m 45.841s W",8,8829a0a5a1fffff,8.258415e+05,The area is a largely residential suburban nei...
27648,5118226,Garden City,40.72677,-73.63430,PPL,US,22612,north,0.0,None,13961.227641,14.0 km north of Garden City,"40 51m 8.96534s N, 73 38m 3.48s W",8,882a101513fffff,7.423544e+05,The area is a primarily residential neighborho...


In [279]:
# Randomly choose a natural disaster from the list
natural_disasters = ['storm', 'flood', 'landslide', 'wild fire']
sample["natural_disaster"] = rng.choice(natural_disasters, size=sample_size)
sample

,geonameid,name,latitude,longitude,feature_code,country_code,population,geospatial_prep,bearing,between_object,distance,target_location_description,target_center,h3_resolution,target_cell,target_area,target_area_description,natural_disaster
26754,4745272,Baileys Crossroads,38.85039,-77.12970,PPL,US,23643,north,0.0,None,1285.888178,1.3 km north of Baileys Crossroads,"38 51m 43.1037s N, 77 7m 46.92s W",7,872aa84c8ffffff,5.009619e+06,The area is a moderately dense urban and subur...,flood
27996,5253352,Fond du Lac,43.77500,-88.43883,PPLA2,US,42933,northwest,315.0,None,9757.568612,9.8 km northwest of Fond du Lac,"43 50m 13.4379s N, 88 31m 28.6298s W",6,86275c367ffffff,3.755665e+07,"The area is a mix of rural farmland, wildlife ...",landslide
26445,4529987,Bartlesville,36.74731,-95.98082,PPLA2,US,36595,west,270.0,None,5334.691803,5.3 km west of Bartlesville,"36 44m 50.262s N, 96 2m 25.9994s W",6,8626ee157ffffff,3.866924e+07,The area is a mix of rural farmland and small ...,storm
28338,5378538,Oakland,37.80437,-122.27080,PPLA2,US,419267,south,180.0,None,82456.334839,82.5 km south of Oakland,"37 3m 41.1298s N, 122 16m 14.88s W",5,85283417fffffff,2.652749e+08,The area is a coastal region in California fea...,wild fire
25834,4165565,Naples,26.14234,-81.79596,PPLA2,US,21512,south,180.0,None,1833.321005,1.8 km south of Naples,"26 7m 32.8523s N, 81 47m 45.456s W",5,854413c7fffffff,2.333873e+08,The area is a coastal region of Southwest Flor...,landslide
28266,5364199,Ladera Ranch,33.57086,-117.63561,PPL,US,22980,between,NaN,"{'adminCode1': 'CA', 'lng': '-117.672', 'dista...",NaN,between Ladera Ranch and Mission Viejo,"33 35m 7.584s N, 117 39m 13.698s W",7,8729a5492ffffff,5.811146e+06,The area is a suburban residential zone center...,flood
28649,5656882,Helena,46.59271,-112.03611,PPLA,US,32091,north,0.0,None,15464.323781,15.5 km north of Helena,"46 43m 54.5606s N, 112 2m 9.996s W",8,882792756dfffff,7.022935e+05,The area is a suburban residential neighborhoo...,storm
27445,5055787,Kirksville,40.19475,-92.58325,PPLA2,US,17520,west,270.0,None,7627.019261,7.6 km west of Kirksville,"40 11m 40.9753s N, 92 40m 22.1546s W",5,85267223fffffff,2.509962e+08,The area is a largely rural region in northeas...,wild fire
28202,5344994,East Los Angeles,34.02390,-118.17202,PPL,US,126496,east,90.0,None,27026.110986,27.0 km east of East Los Angeles,"34 1m 24.7867s N, 117 52m 45.841s W",8,8829a0a5a1fffff,8.258415e+05,The area is a largely residential suburban nei...,flood
27648,5118226,Garden City,40.72677,-73.63430,PPL,US,22612,north,0.0,None,13961.227641,14.0 km north of Garden City,"40 51m 8.96534s N, 73 38m 3.48s W",8,882a101513fffff,7.423544e+05,The area is a primarily residential neighborho...,wild fire


In [280]:
# Invoke LLM to generate report of the natural disaster
def generate_disaster_report(description, disaster, target_location_description):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"Please provide a report of a {disaster} which happened {target_location_description}. The area where the {disaster} occurred is described as: {description}",
                    },
                ],
            }
        ],
    )
    return response.choices[0].message.content
sample["disaster_report"] = sample.apply(
    lambda x: generate_disaster_report(
        description=x["target_area_description"],
        disaster=x["natural_disaster"],
        target_location_description=x["target_location_description"]
    ),
    axis=1
)

In [281]:
sample

,geonameid,name,latitude,longitude,feature_code,country_code,population,geospatial_prep,bearing,between_object,distance,target_location_description,target_center,h3_resolution,target_cell,target_area,target_area_description,natural_disaster,disaster_report
26754,4745272,Baileys Crossroads,38.85039,-77.12970,PPL,US,23643,north,0.0,None,1285.888178,1.3 km north of Baileys Crossroads,"38 51m 43.1037s N, 77 7m 46.92s W",7,872aa84c8ffffff,5.009619e+06,The area is a moderately dense urban and subur...,flood,"## Flood Report: Arlington, Virginia - 1.3km N..."
27996,5253352,Fond du Lac,43.77500,-88.43883,PPLA2,US,42933,northwest,315.0,None,9757.568612,9.8 km northwest of Fond du Lac,"43 50m 13.4379s N, 88 31m 28.6298s W",6,86275c367ffffff,3.755665e+07,"The area is a mix of rural farmland, wildlife ...",landslide,"## Landslide Report: Northwest of Fond du Lac,..."
26445,4529987,Bartlesville,36.74731,-95.98082,PPLA2,US,36595,west,270.0,None,5334.691803,5.3 km west of Bartlesville,"36 44m 50.262s N, 96 2m 25.9994s W",6,8626ee157ffffff,3.866924e+07,The area is a mix of rural farmland and small ...,storm,## Storm Report: Severe Thunderstorm - West of...
28338,5378538,Oakland,37.80437,-122.27080,PPLA2,US,419267,south,180.0,None,82456.334839,82.5 km south of Oakland,"37 3m 41.1298s N, 122 16m 14.88s W",5,85283417fffffff,2.652749e+08,The area is a coastal region in California fea...,wild fire,## Wildfire Incident Report: Año Nuevo Fire - ...
25834,4165565,Naples,26.14234,-81.79596,PPLA2,US,21512,south,180.0,None,1833.321005,1.8 km south of Naples,"26 7m 32.8523s N, 81 47m 45.456s W",5,854413c7fffffff,2.333873e+08,The area is a coastal region of Southwest Flor...,landslide,"## Landslide Report: Coastal Naples, Florida -..."
28266,5364199,Ladera Ranch,33.57086,-117.63561,PPL,US,22980,between,NaN,"{'adminCode1': 'CA', 'lng': '-117.672', 'dista...",NaN,between Ladera Ranch and Mission Viejo,"33 35m 7.584s N, 117 39m 13.698s W",7,8729a5492ffffff,5.811146e+06,The area is a suburban residential zone center...,flood,## Flood Report: Oso Creek Overflow - Ladera R...
28649,5656882,Helena,46.59271,-112.03611,PPLA,US,32091,north,0.0,None,15464.323781,15.5 km north of Helena,"46 43m 54.5606s N, 112 2m 9.996s W",8,882792756dfffff,7.022935e+05,The area is a suburban residential neighborhoo...,storm,"## Storm Report: Helena North - July 26, 2024 ..."
27445,5055787,Kirksville,40.19475,-92.58325,PPLA2,US,17520,west,270.0,None,7627.019261,7.6 km west of Kirksville,"40 11m 40.9753s N, 92 40m 22.1546s W",5,85267223fffffff,2.509962e+08,The area is a largely rural region in northeas...,wild fire,"## Wildfire Incident Report - Adair County, Mi..."
28202,5344994,East Los Angeles,34.02390,-118.17202,PPL,US,126496,east,90.0,None,27026.110986,27.0 km east of East Los Angeles,"34 1m 24.7867s N, 117 52m 45.841s W",8,8829a0a5a1fffff,8.258415e+05,The area is a largely residential suburban nei...,flood,## Flood Incident Report: Creekside Heights - ...
27648,5118226,Garden City,40.72677,-73.63430,PPL,US,22612,north,0.0,None,13961.227641,14.0 km north of Garden City,"40 51m 8.96534s N, 73 38m 3.48s W",8,882a101513fffff,7.423544e+05,The area is a primarily residential neighborho...,wild fire,## Wildfire Incident Report - Village of Sea C...


In [ ]:
# randomly mention area in promot?
# adopt prompt for disaster report, e.g. length